In [1]:
# load library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submission = pd.read_csv("sample_submission.csv")

**데이터 확인하기**


In [3]:
train.head()

,id,reviews,target
0,0,조아요 처음구입 싸게햇어요,2
1,1,생각보다 잘 안돼요 매지 바른지 하루밖에 안됐는데ㅠㅠ 25천원가량 주고 사기 너무 ...,1
2,2,디자인은괜찮은데 상품이 금이가서 교환했는데 두번째받은상품도 까져있고 안쪽에 금이가져...,2
3,3,기전에 이 제품말고 이마트 트레이더스에서만 팔던 프리미엄 제품을 사용했었습니다. 샘...,2
4,4,튼튼하고 손목을 잘 받쳐주네요~,5


In [4]:
train.tail()

,id,reviews,target
24995,24995,일단향이너무너무좋은데지속력만좋다면좋겠지만..워낙저렴해그래도좋아여,5
24996,24996,동일한 업체에서 발송하고 한 상자로 배송되었는데 택배비는 이중 처리되었습니다.,2
24997,24997,향이 좋아요!! 다른 향도 시켜보고 싶어요,4
24998,24998,펼쳐보니 끝부분쯤에 50cm정도 찟어져 있습니다 날짜관계로 구냥 사용합니다,2
24999,24999,호호애미 제품은 늘 믿고 써요,5


train.csv는 id, reviews, target 총 3개의 컬럼으로 이루어져 있습니다.


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       25000 non-null  int64 
 1   reviews  25000 non-null  object
 2   target   25000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 586.1+ KB


In [6]:
train.isnull().sum()

id         0
reviews    0
target     0
dtype: int64

결측값 0개!ㄷㄷ

**데이터 전처리**
<br>
이 대회는 상품 리뷰 텍스트와 별점의 관계성을 파악하는 것이므로 y는 target 컬럼, x는 reviews 컬럼일 것입니다.

In [7]:
# x,y 정의하기
X = train.reviews
y = train.target

<ul>
    <li>워드 임베딩: 텍스트를 숫자 값으로 바꾼 뒤 입력</li>
    <li>CountVectorizer: 입력된 문장을 토큰화하여 토큰의 등장 빈도 벡터로 바꿔주는 기법</li>
    <li>토큰화: 문장을 의미가 있는 최소 단위로 쪼개는 것</li>
</ul>



In [8]:
# countVectorizer 사용해보기
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(train["reviews"])

CountVectorizer()

In [9]:
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(train, test_size=0.25, stratify=train.target)

train_data = train_data.reset_index().drop('index', axis=1)
val_data = val_data.reset_index().drop('index', axis=1)


In [10]:
print("train data:", train_data.shape)
print("val data:", val_data.shape)

train data: (18750, 3)
val data: (6250, 3)


In [11]:
train_X = train_data.reviews
train_y = train_data.target

val_X = val_data.reviews
val_y = val_data.target


In [12]:
train_X = vectorizer.transform(train_X)
val_X = vectorizer.transform(val_X)

In [13]:
print(train_X[10])

  (0, 3580)	1
  (0, 4938)	1
  (0, 13413)	1
  (0, 17164)	1
  (0, 19485)	1
  (0, 29042)	1
  (0, 35639)	1
  (0, 43832)	1
  (0, 48144)	1
  (0, 58516)	1
  (0, 63167)	1
  (0, 66168)	1
  (0, 67766)	1
  (0, 71410)	1


In [14]:
vectorizer.inverse_transform(train_X[10])

[array(['가성비', '거기에', '너무', '대박', '듣던대로', '방수포로', '사용했어요', '아래', '엄청커요',
        '접어서', '짱이네어', '커서', '텐트', '한번'], dtype='<U100')]

In [15]:
#로지스틱 회귀: category 분류
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=500)
model.fit(train_X, train_y)

LogisticRegression(max_iter=500)

In [16]:
# 정확도 판단 함수
def ACCURACY(true, pred):
    score = np.mean(true==pred)
    return score


In [17]:
y_pred = model.predict(val_X)

In [18]:
score = ACCURACY(val_y, y_pred)
print("모델의 정확도:", score)

모델의 정확도: 0.60208


TRAIN 데이터와 VAL 데이터셋 100퍼센트 활용하는 모델


In [19]:
X = train_data.reviews
y = train_data.target

vectorizer = CountVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

In [20]:
model = LogisticRegression(max_iter=500)
model.fit(X,y)

LogisticRegression(max_iter=500)

In [21]:
from sklearn.metrics import accuracy_score

y_pred = model.predict(X[0])
print("예측라벨:", y_pred)
print("실제라벨:", train.target[0])

예측라벨: [5]
실제라벨: 2


In [22]:
vectorizer.inverse_transform(X[0])


[array(['늦지만', '배송은', '사은품도', '있고', '좋아요'], dtype='<U100')]

In [23]:
#예측하기
test_X = test.reviews
test_X_vect = vectorizer.transform(test_X)

In [24]:
pred = model.predict(test_X_vect)
print(pred)

[5 2 5 ... 2 5 1]


In [25]:
submission["target"] = pred
submission.head()

,id,target
0,0,5
1,1,2
2,2,5
3,3,1
4,4,1


In [26]:
submission.to_csv("submission.csv", index=False)